In [1]:
import os
import getpass
import sqlite3

In [23]:
os.environ["LANGSMITH_API_KEY"] = "xx"

In [25]:
# 1) 如果要在 Notebook 中临时输入 OpenAI API key，可用下列方式
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# 2) 构建 SQLite 数据库路径
db_path = os.path.expanduser("~/Desktop/patents/data/patent.db")

# 3) 利用 SQLAlchemy 建立连接引擎
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool
from sqlalchemy import text

engine = create_engine(
    f"sqlite:///{db_path}",
    connect_args={"check_same_thread": False},
    poolclass=StaticPool
)

# 测试一下是否能连接成功
with engine.begin() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    tables = result.fetchall()
    
print(f"Successfully connected to database at {db_path}")

Successfully connected to database at /Users/yuxiangwang/Desktop/patents/data/patent.db


In [27]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase(engine)

print(type(db))

<class 'langchain_community.utilities.sql_database.SQLDatabase'>


In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o",  # 或 "gpt-3.5-turbo"
    temperature=0.0,
)

In [31]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
print("Available tools:")
for t in tools:
    print(" -", t.name, ":", t.description)

Available tools:
 - sql_db_query : Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
 - sql_db_schema : Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
 - sql_db_list_tables : Input is an empty string, output is a comma-separated list of tables in the database.
 - sql_db_query_checker : Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!


In [37]:
from langchain import hub

# 从langchain-hub拉取一个 SQL 问答提示模板
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
print(prompt_template.input_variables)  # 确认需要替换哪些字段

['dialect', 'top_k']


/opt/anaconda3/envs/patent/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [39]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

In [45]:
from langchain import hub
from langgraph.prebuilt import create_react_agent

# 使用 langgraph 提供的 create_react_agent 方法
agent_executor = create_react_agent(
    model=llm, 
    tools=toolkit.get_tools(), 
    prompt=system_message
)


In [53]:
user_question = "数据库里有哪些表？"

events = agent_executor.stream(
    {"messages": [("user", user_question)]}, 
    stream_mode="values"
)

for event in events:
    # 这里 event["messages"] 是本次状态变化产出的对话消息列表
    # 打印或处理最新生成的那一条
    msg = event["messages"][-1]
    msg.pretty_print()

================================ Human Message =================================

数据库里有哪些表？
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_EyI9VRLK3iC8xVlGP2emF2WB)
 Call ID: call_EyI9VRLK3iC8xVlGP2emF2WB
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

applications_claiming_priority, assignees, child_applications, cited_by, claims, classifications, concepts, error_logs, events, external_links, images, inventors, legal_events, non_patent_citations, parent_applications, patent_citations, patents, prior_art_keywords, priority_applications, similar_documents, sqlite_sequence, worldwide_applications
================================== Ai Message ==================================

数据库中有以下表：

1. applications_claiming_priority
2. assignees
3. child_applications
4. cited_by
5. claims
6. classifications
7. concepts
8. error_logs
9. events
10. external_links


In [55]:
help(agent_executor)

Help on CompiledStateGraph in module langgraph.graph.state object:

class CompiledStateGraph(langgraph.graph.graph.CompiledGraph)
 |  CompiledStateGraph(*, builder: langgraph.graph.graph.Graph, **kwargs: Any) -> None
 |
 |  Method resolution order:
 |      CompiledStateGraph
 |      langgraph.graph.graph.CompiledGraph
 |      langgraph.pregel.Pregel
 |      langgraph.pregel.protocol.PregelProtocol
 |      langchain_core.runnables.base.Runnable
 |      typing.Generic
 |      abc.ABC
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  attach_branch(self, start: str, name: str, branch: langgraph.graph.graph.Branch, *, with_reader: bool = True) -> None
 |
 |  attach_edge(self, starts: Union[str, Sequence[str]], end: str) -> None
 |
 |  attach_node(self, key: str, node: Optional[langgraph.graph.state.StateNodeSpec]) -> None
 |
 |  get_input_schema(self, config: Optional[langchain_core.runnables.config.RunnableConfig] = None) -> type[pydantic.main.BaseModel]
 |      Get a pydantic mo